In [0]:
'''from google.colab import drive
drive.mount('/content/gdrive')'''

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
import os
import time
import spacy

In [0]:
!git clone https://github.com/saadmoumad/NLP-on-Morocco-s-King-Speech

In [0]:
#Scraping Discours from maroc.ma 
!python3 NLP-on-Morocco-s-King-Speech/DiscourScraping.py
#Alredy pushed with git-repo

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
#installing Spacy for text preprocessing
!pip3 install spacy

In [0]:
#Downloading French model
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [0]:
nlp = spacy.load("/usr/local/lib/python3.6/dist-packages/fr_core_news_sm/fr_core_news_sm-2.2.5")

In [0]:
data_dir = 'NLP-on-Morocco-s-King-Speech/Disc_Text'
files_name = os.listdir(data_dir)
files_name.sort()

In [0]:
len(files_name)

205

In [0]:
#Passing a Spacy.doc object and returning a list of string Token 
def get_token_list(doc):
    l = []
    for token in doc:
        l.append(str(token))
    return l

In [0]:
#Return a set objet of unique token in all 'Discours'
def get_vocab_set(files_name):
    s = set()
    for file in files_name[1:]: #exclu .DS_Store file
        text = open(os.path.join(data_dir, file), 'rb').read().decode(encoding='utf-8')
        doc = nlp(text)
        token_list = get_token_list(doc)
        s_next = set(token_list)
        s = s.union(s_next.difference(s))
    return sorted(s)

In [0]:
vocab_set = get_vocab_set(files_name)

In [0]:
len(vocab_set)

15701

In [0]:
#Convert Text (as Tokens) to int
def text_as_int(files_name, word2idx):
    text_int = []
    for file in files_name[1:]: #exclu .DS_Store file
          text = open(os.path.join(data_dir, file), 'rb').read().decode(encoding='utf-8')
          doc = nlp(text)
          token_list = get_token_list(doc)
          text_int += [word2idx[word] for word in token_list]
    return text_int

In [0]:
# Creating a mapping from unique Token to indices
word2idx = {u:i for i, u in enumerate(vocab_set)}
idx2word = np.array(vocab_set)

In [0]:
text_as_int = np.array(text_as_int(files_name, word2idx))

In [0]:
len(text_as_int)

447294

In [0]:
seq_length = 30
examples_per_epoch = len(text_as_int)//(seq_length+1)

In [0]:
words_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = words_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for i in dataset.take(2):
  print(i)

(<tf.Tensor: shape=(30,), dtype=int64, numpy=
array([   13,  1368, 14926,   874,    13,  1809,  8944,  1758,  6738,
        8944,   653, 13369, 13808,  9000,  1768,    26,  1928,  1026,
        6738,  1982,   766, 10756, 14926,  8944, 14193,  8464, 10407,
        9984, 14859, 11677])>, <tf.Tensor: shape=(30,), dtype=int64, numpy=
array([ 1368, 14926,   874,    13,  1809,  8944,  1758,  6738,  8944,
         653, 13369, 13808,  9000,  1768,    26,  1928,  1026,  6738,
        1982,   766, 10756, 14926,  8944, 14193,  8464, 10407,  9984,
       14859, 11677,    26])>)
(<tf.Tensor: shape=(30,), dtype=int64, numpy=
array([ 1928,  1419,  9000,  1873,  1155,  1189,    26, 11787,   874,
        9215,  6583,  5320, 13413,  8295,  1495,    26, 11018,  4414,
        9000,  8969,  5320,  8944,  7547,  1435,  2191,    26,  9997,
       13410,  7751,  5320])>, <tf.Tensor: shape=(30,), dtype=int64, numpy=
array([ 1419,  9000,  1873,  1155,  1189,    26, 11787,   874,  9215,
        6583,  5320, 1341

In [0]:
BATCH_SIZE = 32
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the Tokens vocab
vocab_size = len(vocab_set)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab_set),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           4019456   
_________________________________________________________________
gru (GRU)                    (32, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (32, None, 15701)         16093525  
Total params: 24,051,285
Trainable params: 24,051,285
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [0]:
#Checkpoints for model training
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=20

In [0]:
#fit Data to model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
450/450 [==============================] - 53s 119ms/step - loss: 6.2119
Epoch 2/20
450/450 [==============================] - 53s 117ms/step - loss: 5.2796
Epoch 3/20
450/450 [==============================] - 53s 118ms/step - loss: 4.8994
Epoch 4/20
450/450 [==============================] - 53s 118ms/step - loss: 4.2236
Epoch 5/20
450/450 [==============================] - 54s 119ms/step - loss: 3.7160
Epoch 6/20
450/450 [==============================] - 53s 118ms/step - loss: 3.2289
Epoch 7/20
450/450 [==============================] - 53s 118ms/step - loss: 2.7709
Epoch 8/20
450/450 [==============================] - 53s 119ms/step - loss: 2.3841
Epoch 9/20
450/450 [==============================] - 53s 118ms/step - loss: 2.0789
Epoch 10/20
450/450 [==============================] - 53s 117ms/step - loss: 1.8388
Epoch 11/20
450/450 [==============================] - 53s 117ms/step - loss: 1.6509
Epoch 12/20
450/450 [==============================] - 53s 118ms/step - lo

In [0]:
#Rebuild with diff batch size and restore the weights from the latest checkpoint.
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  doc = nlp(start_string)
  start_token = get_token_list(doc)
  num_generate = 1000

  input_eval = [word2idx[s] for s in start_token]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []
  temperature = 1.0
  model.reset_states()
  
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      # using a categorical distribution to predict the token returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2word[predicted_id])

  return (start_string + ' '.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"SM le Roi"))

SM le RoiMohammed VI , que Dieu L' assiste , a adressé samedi un discours à la Nation à l’ occasion de la Fête du Trône , qui coïncide cette année avec le 17-ème pour exprimer mes sincères remerciements et ma considération à l’ organisation et ce faire en son sein de l' environnement et du développement , nous nous pouvons en nous engageons ensemble et dans le cadre de l' Initiative que Nous source de production , en prenant les délais les plus ouvertes . 
 C' est pourquoi Nous ne cessons d' exprimer , des négociations qui ont été adoptées . 
 Nous exprimons également ces fondations à la concertation , aux organisations de la société civile , sous l’ impulsion du secteur public dans notre pays . A ce titre , Nous est déterminé à renforcer et à veiller résolument sur la régionalisation avancée , dans la mesure où les progrès sont reconnus au voisinage sont un cadre pour les doter d' entre eux , et le soutien public et le développement social de notre pays . 
 Mes frères Chefs d’ Etat au